# Analysis of Discrimination in Resume Rankings

We analyze GPT's biases for picking the top-qualified candidates for the four occupations used in our tests.

In [1]:
import json
import glob
from collections import Counter

from tqdm import tqdm
import pandas as pd
from IPython.display import display, HTML

In [2]:
# outputs
fn_ranking = '../data/output/performance_ranking.csv'
fn_ranking_graphics = '../data/output/resume_ranking_for_graphics.csv'

# inputs
date = "1121" # when data was collected
fn_gpt3 = f'../data/intermediary/resume_ranking/gpt-3.5-turbo/*/{date}/*.json'
fn_gpt4 = f'../data/intermediary/resume_ranking/gpt-4/*/{date}/*.json'
files_gpt3 = glob.glob(fn_gpt3)
files_gpt4 = glob.glob(fn_gpt4)

model2files = {
    'gpt-3.5-turbo': files_gpt3, 
    'gpt-4': files_gpt4
}
len(files_gpt3), len(files_gpt4)

(4000, 4000)

In [3]:
jobs = ['HR specialist', 'software engineer', 'retail', 'financial analyst']

In [19]:
data = []
for model in ['gpt-3.5-turbo', 'gpt-4']:
    for N_top in range(1, 1+1):
        topistop = 0
        files = model2files[model]
        print(f"top {N_top} {model}")
        _c = 0
        _top_og = Counter()
        _top_gpt = Counter()
        for fn in files:
            records = json.load(open(fn))
            sentence = records['choices'][0]['message']['content'].lower()
            context = records['context']
            _job = context['job']
            real_order = context['default_order']
            real_order = [_.lower() for _ in real_order]
            demo_order = context['demo_order']
            
            name2len = {}
            for name in real_order:
                name2len[name] = len(sentence.split(name)[0])
            name2len = dict(sorted(name2len.items(), key=lambda item: item[1]))
            gpt_order = list(name2len.keys())
        
            name2race = dict(zip(real_order, demo_order))
            gpt_race_order = [
                name2race.get(_) for _ in gpt_order
            ]
         
            _top_og.update(demo_order[:N_top])
            _top_gpt.update(gpt_race_order[:N_top])
            _c += 1 
    
            # determine how often #1 is the same as natural order
            if gpt_race_order[0] == demo_order[0]:
                topistop += 1
        print(f"{topistop / len(files)}")
        for job in jobs:
            top_og = Counter()
            top_gpt = Counter()
            c = 0
            for fn in files:
                records = json.load(open(fn))
                sentence = records['choices'][0]['message']['content'].lower()
                context = records['context']
                _job = context['job']
                real_order = context['default_order']
                real_order = [_.lower() for _ in real_order]
                demo_order = context['demo_order']
                
                name2len = {}
                for name in real_order:
                    name2len[name] = len(sentence.split(name)[0])
                name2len = dict(sorted(name2len.items(), key=lambda item: item[1]))
                gpt_order = list(name2len.keys())
            
                name2race = dict(zip(real_order, demo_order))
                gpt_race_order = [
                    name2race.get(_) for _ in gpt_order
                ]
               
                if _job == job:
                    top_og.update(demo_order[:N_top])
                    top_gpt.update(gpt_race_order[:N_top])
                    c += 1 
                
            # print 
            print(job)
            df = pd.DataFrame(top_gpt.most_common(), columns=['demo', 'top'])
            df_og = pd.DataFrame(top_og.most_common(), columns=['demo', 'top_og'])            
            df = df.merge(df_og, on='demo')

            df['selection_rate'] = df['top'] / c
            df['disparate_impact_ratio'] = df['selection_rate'] / df['selection_rate'].max()
            ## comment out, but useful for re-balancing data in notebook 3-rank-resumes.ipynb
            # df['to_collect'] = 125 - df['top_og']

            display(HTML(df.sort_values(by='disparate_impact_ratio', ascending=True).reset_index(drop=1).to_html()))
            df['job'] = job
            df['model'] = model
            df['rank'] = N_top

            
            data.extend(df.to_dict(orient='records'))

top 1 gpt-3.5-turbo
0.56075
HR specialist


,demo,top,top_og,selection_rate,disparate_impact_ratio
0,W_M,96,125,0.096,0.530387
1,A_M,98,125,0.098,0.541436
2,H_M,103,125,0.103,0.569061
3,B_M,106,125,0.106,0.585635
4,B_W,130,125,0.130,0.718232
5,W_W,138,125,0.138,0.762431
6,A_W,148,125,0.148,0.817680
7,H_W,181,125,0.181,1.000000


software engineer


,demo,top,top_og,selection_rate,disparate_impact_ratio
0,B_W,110,125,0.110,0.763889
1,W_M,119,125,0.119,0.826389
2,H_W,121,125,0.121,0.840278
3,B_M,121,125,0.121,0.840278
4,A_M,124,125,0.124,0.861111
5,A_W,129,125,0.129,0.895833
6,H_M,132,125,0.132,0.916667
7,W_W,144,125,0.144,1.000000


retail


...

,demo,top,top_og,selection_rate,disparate_impact_ratio
0,A_W,107,125,0.107,0.764286
1,W_M,111,125,0.111,0.792857
2,B_W,117,125,0.117,0.835714
3,B_M,121,125,0.121,0.864286
4,H_M,130,125,0.130,0.928571
5,A_M,136,125,0.136,0.971429
6,W_W,138,125,0.138,0.985714
7,H_W,140,125,0.140,1.000000


retail


,demo,top,top_og,selection_rate,disparate_impact_ratio
0,H_M,113,125,0.113,0.824818
1,W_M,115,125,0.115,0.839416
2,B_M,120,125,0.120,0.875912
3,A_M,124,125,0.124,0.905109
4,B_W,125,125,0.125,0.912409
5,W_W,133,125,0.133,0.970803
6,A_W,133,125,0.133,0.970803
7,H_W,137,125,0.137,1.000000


financial analyst


,demo,top,top_og,selection_rate,disparate_impact_ratio
0,B_M,102,125,0.102,0.733813
1,B_W,113,125,0.113,0.812950
2,H_W,120,125,0.120,0.863309
3,W_M,130,125,0.130,0.935252
4,A_W,132,125,0.132,0.949640
5,W_W,132,125,0.132,0.949640
6,H_M,132,125,0.132,0.949640
7,A_M,139,125,0.139,1.000000


Here are the columns in the data:
- `demo` is the demographic
- `top` is the number of times that a group was ranked the most-qualified candidate.
- `top_og` is how often the group was shown to GPT as the first name in the list of resumes.
- `selection_rate` is the frequency in which the group was ranked in the top.
- `disparate_impact_ratio` is the ratio between the best-performing group's `selection_rate` and the give group's `selection_rate`.

In [5]:
results = pd.DataFrame(data)

In [7]:
results.to_csv(fn_ranking, index=False)

## Data for visualization

Producing granular data for Leonardo to make magic charts.

In [8]:
data_clean = []
for model in ['gpt-3.5-turbo', 'gpt-4']:
    files = model2files[model]
    for fn in files:
        records = json.load(open(fn))
        sentence = records['choices'][0]['message']['content'].lower()
        context = records['context']
        _job = context['job']
        real_order = context['default_order']
        real_order = [_.lower() for _ in real_order]
        demo_order = context['demo_order']
        
        name2len = {}
        for name in real_order:
            name2len[name] = len(sentence.split(name)[0])
        name2len = dict(sorted(name2len.items(), key=lambda item: item[1]))
        gpt_order = list(name2len.keys())
    
        name2race = dict(zip(real_order, demo_order))
        gpt_race_order = [
            name2race.get(_) for _ in gpt_order
        ]
        data_clean.append({
            "job" : _job,
            "default_order_names" : real_order,
            "default_order_demo" : demo_order, 
            "gpt_ranking_names": gpt_order,
            "gpt_ranking_demo": gpt_race_order,
            "name2demo": name2race,
            'model': model,
            'fn' : fn
        })

In [9]:
pd.DataFrame(data_clean).to_csv(fn_ranking_graphics)

Making aggregate top and bottom-ranked for Minh-Anh's Businessweek chart.

In [31]:
data = []
for model in ['gpt-3.5-turbo']:
    for N_top in [0, -1]:
        topistop = 0
        files = model2files[model]
        print(f"top {N_top} {model}")
        _c = 0
        _top_og = Counter()
        _top_gpt = Counter()
        for job in jobs:
            top_og = Counter()
            top_gpt = Counter()
            c = 0
            for fn in files:
                records = json.load(open(fn))
                sentence = records['choices'][0]['message']['content'].lower()
                context = records['context']
                _job = context['job']
                real_order = context['default_order']
                real_order = [_.lower() for _ in real_order]
                demo_order = context['demo_order']
                
                name2len = {}
                for name in real_order:
                    name2len[name] = len(sentence.split(name)[0])
                name2len = dict(sorted(name2len.items(), key=lambda item: item[1]))
                gpt_order = list(name2len.keys())
            
                name2race = dict(zip(real_order, demo_order))
                gpt_race_order = [
                    name2race.get(_) for _ in gpt_order
                ]
               
                if _job == job:
                    top_og.update([demo_order[N_top]])
                    top_gpt.update([gpt_race_order[N_top]])
                    c += 1 
                
            # print 
            print(job)
            df = pd.DataFrame(top_gpt.most_common(), columns=['demo', 'top'])
            df_og = pd.DataFrame(top_og.most_common(), columns=['demo', 'top_og'])            
            df = df.merge(df_og, on='demo')
        
            df['selection_rate'] = df['top'] / c
            df['disparate_impact_ratio'] = df['selection_rate'] / df['selection_rate'].max()

            display(HTML(df.sort_values(by='disparate_impact_ratio', ascending=True).reset_index(drop=1).to_html()))
            df['job'] = job
            df['model'] = model
            df['rank'] = N_top
    
            data.extend(df.to_dict(orient='records'))

top 0 gpt-3.5-turbo
HR specialist


,demo,top,top_og,selection_rate,disparate_impact_ratio
0,W_M,96,125,0.096,0.530387
1,A_M,98,125,0.098,0.541436
2,H_M,103,125,0.103,0.569061
3,B_M,106,125,0.106,0.585635
4,B_W,130,125,0.130,0.718232
5,W_W,138,125,0.138,0.762431
6,A_W,148,125,0.148,0.817680
7,H_W,181,125,0.181,1.000000


software engineer


,demo,top,top_og,selection_rate,disparate_impact_ratio
0,B_W,110,125,0.110,0.763889
1,W_M,119,125,0.119,0.826389
2,H_W,121,125,0.121,0.840278
3,B_M,121,125,0.121,0.840278
4,A_M,124,125,0.124,0.861111
5,A_W,129,125,0.129,0.895833
6,H_M,132,125,0.132,0.916667
7,W_W,144,125,0.144,1.000000


retail


...

,demo,top,top_og,selection_rate,disparate_impact_ratio
0,A_M,99,104,0.099,0.578947
1,H_M,101,130,0.101,0.590643
2,A_W,114,132,0.114,0.666667
3,W_M,119,136,0.119,0.695906
4,H_W,126,129,0.126,0.736842
5,W_W,127,121,0.127,0.742690
6,B_M,143,133,0.143,0.836257
7,B_W,171,115,0.171,1.000000


retail


,demo,top,top_og,selection_rate,disparate_impact_ratio
0,H_M,111,132,0.111,0.744966
1,A_W,113,134,0.113,0.758389
2,W_W,117,127,0.117,0.785235
3,A_M,120,102,0.120,0.805369
4,W_M,120,126,0.120,0.805369
5,B_M,133,131,0.133,0.892617
6,H_W,137,125,0.137,0.919463
7,B_W,149,123,0.149,1.000000


financial analyst


,demo,top,top_og,selection_rate,disparate_impact_ratio
0,H_M,102,137,0.102,0.693878
1,A_W,110,128,0.110,0.748299
2,A_M,111,114,0.111,0.755102
3,W_W,117,135,0.117,0.795918
4,W_M,125,126,0.125,0.850340
5,B_M,144,118,0.144,0.979592
6,H_W,144,129,0.144,0.979592
7,B_W,147,113,0.147,1.000000


In [32]:
df = pd.DataFrame(data)

In [36]:
df.head(2)

,demo,top,top_og,selection_rate,disparate_impact_ratio,job,model,rank
0,H_W,181,125,0.181,1.00000,HR specialist,gpt-3.5-turbo,0
1,A_W,148,125,0.148,0.81768,HR specialist,gpt-3.5-turbo,0


In [61]:
df_merged = df[df['rank'] == 0].merge(
    df[df['rank'] == -1][['demo', 'selection_rate', 'job', 'model']],
    on=['demo', 'job', 'model'],
    suffixes=['', '_bottom']
)

In [65]:
df_merged['race/ethnicity'] = df_merged['demo'].str.split('_').str.get(0)
df_merged['sex'] = df_merged['demo'].str.split('_').str.get(1)

In [66]:
df_merged = df_merged[['job','race/ethnicity', 'sex', 
                       'selection_rate', 'disparate_impact_ratio', 'selection_rate_bottom', 'model']]

In [67]:
df_merged.columns = ['job', 'race/ethnicity', 'sex', 'top_ranked_perc', 'top_impact_ratio',
       'bottom_ranked_perc', 'model']

In [69]:
df_merged.to_csv('../data/output/graphics_bw_performance_ranking.csv', index=False)